# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 

The architecture follows Encoder-Decoder design, a combination of deep convolutional nets for image classification with recurrent networks for sequence modeling, to create a single network that generates descriptions of images.

**CNN Encoder** - deep convolutional network that takes input image, extracts convolutional features and embeds these in a fixed-length vector. In particular pre-trained Resnet model is used as CNN encoder where the last hidden layer is used as an input to the RNN decoder that generates sentences.

**RNN Decoder** is then used to maximize the probability of the correct description given the image - p(S(t)|I, S(0), ..., S(t−1)). In particular, I have used LSTM net which has shown state-of-the art performance on sequence tasks such
as translation. The LSTM model is trained to predict each word of the sentence after it has seen the image as well as all preceding words as defined by p(S(t)|I, S(0), ..., S(t−1)).

The RNN Decoder consists of following layers:

- Embedding layer - nn.Embedding(vocab_size, embed_size) to convert caption tokens to vector representations

- LSTM layer - nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True, dropout=0.5) to process image features and sequence of tokens 

- Fully connected output layer - nn.Linear(hidden_size, vocab_size) - to produce scores for each token in vocabulary. The output is then fed to nn.CrossEntropyLoss() that applies LogSoftmax on scores and then calculates cross entropy loss.

**Decoder.forward** 

- takes extracted image features and captions
- converts captions via embedding layer
- concatenates image features with token embeddings and feeds concatenated data to LSTM layer that processes the entire token sequence, the output of the LSTM at time t−1 is fed to the LSTM at time t
- output of LSTM layer is then fed to fully connected layer to calculate scores.


    def forward(self, features, captions):
        # embed tokens in vector space
        embeddings = self.embedding(captions)

        # append image as first input
        inputs = torch.cat((features.unsqueeze(1), embeddings), 1)

        lstm_out, lstm_states = self.lstm(inputs)
        output = self.linear(self.dropout(lstm_out))
        return output[:, 1:]


**Decoder.sample**

- takes extracted image features as passes it into LSTM layer, image is fed to the network only once
- output of LSTM layer is fed to fully connected layer to produce scores for each token 
- the index (into the vocabulary) of the predicted token is then determined as following -> predicted = outputs.max(1)[1]
- predicted token gets converted to its corresponding embedding which is then used as input again into LSTM layer, continuing like this until we sample the special end-of-sentence token or some maximum length


    def sample(self, features, states=None, max_len=20):
        " accepts pre-processed image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "

        """Samples captions for given image features (Greedy search)."""
        output_ids = []

        inputs = features

        for i in range(max_len):
            # pass data through recurrent network
            hiddens, states = self.lstm(inputs, states)
            outputs = self.linear(hiddens.squeeze(1))

            # find maximal predictions
            predicted = outputs.data.max(1)[1]

            # append results from given step to global results
            output_ids.append(predicted.item())

            # prepare chosen words for next decoding step
            embeddings = self.embedding(predicted)
            inputs = embeddings.unsqueeze(1)

        return output_ids


Hyperparams values - i have used hyperparams values as per article: Vinyals... - Show and Tell: A Neural Image Caption Generator (https://arxiv.org/abs/1411.4555)

### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

Resize - most neural networks expect the images of a fixed size
RandomCrop - to crop from image randomly - data augmentation, helps to train network for the unseen variations of images
RandomHorizontalFlip - horizontally flip the given image randomly with a probability of 0.5 - data augmentation
ToTensor - to convert the numpy images to torch images as we need to swap axes
Normalize - normalizes an tensor image with mean and standard deviation, given mean: (R, G, B) and std: (R, G, B), will normalize each channel of the torch - channel = (channel - mean) / std


### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

The encoder-decoder network loss is minimized w.r.t. all the parameters of the LSTM, output-fully connected layer of decoder, decoder word embeddings, and the top layer of the image CNN encoder. In other words, encoder-decoder loss network is trained using all parameters except the poaramneters of resnet backbone - pre-trained image feature extraction layer (end of section 3.1 - Vinyals... - Show and Tell: A Neural Image Caption Generator).

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

**Adam** - Adaptive Moment estimation - combines the ideas of Momentum optimization and RMSProp: just like Momentum optimization it keeps track of an exponentially decaying average of past gradients, and just like RMSProp it keeps track of an exponentially decaying average of past squared gradients.

Since Adam is an adaptive learning rate algorithm (like AdaGrad and RMSProp), it requires less tuning of the learning rate hyperparameter. 

Consulting the research paper - Xu, Ba.. - Show, Attend and Tell: Neural Image Caption Generation with Visual Attention -> for the Flickr8k dataset, they found that RMSProp worked best, while for Flickr30k/MS COCO dataset they used the Adam algorithm. 

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
sys.path.append('/usr/local/lib/python3.6/dist-packages')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
learning_rate = 1e-3

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=0.56s)
creating index...


  0%|          | 844/414113 [00:00<00:48, 8434.14it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:41<00:00, 9941.81it/s]
/home/slavo/.local/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [2]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions[:,:-1])
        
        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions[:,1:].contiguous().view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 4.2550, Perplexity: 70.4547
Epoch [1/3], Step [200/3236], Loss: 3.8210, Perplexity: 45.64981
Epoch [1/3], Step [300/3236], Loss: 3.5383, Perplexity: 34.40772
Epoch [1/3], Step [400/3236], Loss: 3.3319, Perplexity: 27.9922
Epoch [1/3], Step [500/3236], Loss: 3.3637, Perplexity: 28.89674
Epoch [1/3], Step [600/3236], Loss: 3.2127, Perplexity: 24.8450
Epoch [1/3], Step [700/3236], Loss: 3.0510, Perplexity: 21.1375
Epoch [1/3], Step [800/3236], Loss: 3.2305, Perplexity: 25.2921
Epoch [1/3], Step [900/3236], Loss: 3.2669, Perplexity: 26.2287
Epoch [1/3], Step [1000/3236], Loss: 2.9699, Perplexity: 19.4897
Epoch [1/3], Step [1100/3236], Loss: 3.1372, Perplexity: 23.0403
Epoch [1/3], Step [1200/3236], Loss: 2.9529, Perplexity: 19.1617
Epoch [1/3], Step [1300/3236], Loss: 2.7113, Perplexity: 15.0492
Epoch [1/3], Step [1400/3236], Loss: 2.9642, Perplexity: 19.3784
Epoch [1/3], Step [1500/3236], Loss: 3.3693, Perplexity: 29.0594
Epoch [1/3], Step [1600/3236], 

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.

**Show Attend Tell Model**

In addition to the above Show and Tell model I have also implemented Show Attend Tell model with attention as per article: Show, Attend and Tell: Xu, Ba... - Neural Image Caption Generation with Visual Attention (https://arxiv.org/abs/1502.03044). 

Show Attend Tell network architecture:

**CNN Encoder**

- encoder extract features from a lower convolutional layer unlike show and tell model (above) which instead used a fully connected layer. This allows the decoder to selectively focus on certain parts of an image by selecting a subset of all the feature vectors. To extract image features (annotation vectors a(i)) I have used pretrained VGGnet on ImageNet without finetuning. In particular I use the 14×14×512 feature map of the fourth convolutional layer before max pooling. 

**RNN Encoder**

- LSTM network produces a caption by generating one word at every time step conditioned on a context vector, the previous hidden state and the previously generated words


- the context vector z(t) is a dynamic representation of the relevant part of the image input at time t. It is computed by an attention model that computes z(t) from the annotation vectors a(i) corresponding to the features extracted at different image locations. For each location i, the attention model generates a positive weight α(i) which can be interpreted either as the probability that location i is the right place to focus for producing the next word, or as the relative importance to give to location i in blending the a(i)’s together. Attention model uses a multilayer perceptron conditioned on the previous hidden state h(t−1). Deterministic soft attention weighted annotation vector is then computed as following φ ({a(i)} ; {α(i)}) = SUM α(i)a(i)



    def attention_layer(self, features, context_encode, hidden):
        h_att = F.tanh(context_encode + self.weight_hh(hidden).unsqueeze(1).expand_as(context_encode))
        out_att = torch.bmm(h_att, self.weight_att.unsqueeze(0).expand(h_att.size(0), self.weight_att.size(0),
                                                                       self.weight_att.size(1))).squeeze(2)
        alpha = F.softmax(out_att)
        context = (features * alpha.unsqueeze(2).expand_as(features)).mean(1).squeeze(1)
        return context, alpha



- deep output layer is used to compute the output word probability given the LSTM state, the context vector and the previous word:


    def output_layer(self, context, hidden, prev=None):
        context = self.context2out(context)
        hidden = self.hidden2tout(hidden)
        out = self.classifier(context + hidden)
        return out


In [3]:
import math
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
sys.path.append('/usr/local/lib/python3.6/dist-packages')
from pycocotools.coco import COCO
from data_loader import get_loader
from show_attend_tell import ShowAttendTellModel

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity
learning_rate = 1e-3

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the ShowAttendtell model
model = ShowAttendTellModel(hidden_size, embed_size, vocab_size, embed_size)

# Move models to GPU if CUDA is available.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function.
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = filter(lambda p: p.requires_grad, model.parameters())

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, lr=learning_rate)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 894/414113 [00:00<00:46, 8932.62it/s]

Done (t=0.50s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:41<00:00, 9865.91it/s]


In [4]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        lengths = [len(cap)-1 for cap in captions]
        
        # Zero the gradients.
        model.zero_grad()
        
        # Pass the inputs through the model.
        outputs = model(images, captions[:, :-1], lengths)
        
        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions[:,1:].contiguous().view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
    # Save the weights.
    if epoch % save_every == 0:
        torch.save(model.state_dict(), os.path.join('./models', 'show-attend-tell-%d.pkl' % epoch))

# Close the training log file.
f.close()

/home/slavo/Dev/CVND---Image-Captioning-Project/show_attend_tell.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  alpha = F.softmax(out_att)


Epoch [1/3], Step [100/3236], Loss: 5.3969, Perplexity: 220.7238
Epoch [1/3], Step [200/3236], Loss: 5.3546, Perplexity: 211.5888
Epoch [1/3], Step [300/3236], Loss: 5.2501, Perplexity: 190.5895
Epoch [1/3], Step [400/3236], Loss: 5.3051, Perplexity: 201.3531
Epoch [1/3], Step [500/3236], Loss: 5.4760, Perplexity: 238.8868
Epoch [1/3], Step [600/3236], Loss: 5.6385, Perplexity: 281.0350
Epoch [1/3], Step [700/3236], Loss: 5.4269, Perplexity: 227.4425
Epoch [1/3], Step [800/3236], Loss: 5.2068, Perplexity: 182.5061
Epoch [1/3], Step [900/3236], Loss: 5.2547, Perplexity: 191.4562
Epoch [1/3], Step [1000/3236], Loss: 5.3321, Perplexity: 206.8783
Epoch [1/3], Step [1100/3236], Loss: 5.3639, Perplexity: 213.5505
Epoch [1/3], Step [1200/3236], Loss: 5.2444, Perplexity: 189.5078
Epoch [1/3], Step [1300/3236], Loss: 5.3248, Perplexity: 205.3773
Epoch [1/3], Step [1400/3236], Loss: 5.2921, Perplexity: 198.7676
Epoch [1/3], Step [1500/3236], Loss: 5.3058, Perplexity: 201.5036
Epoch [1/3], Step [